In [ ]:
#Importing Dependencies
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#Downloading all thhe data files into the colab
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
!wget http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

In [ ]:
#Copy the downloaded files into the project folder
!mkdir MNIST_Fashion
!cp *.gz MNIST_Fashion

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_Fashion/")

In [ ]:
#Training Parameters
learning_rate = 0.0002
batch_size = 128
epochs = 40000

In [ ]:
#Network Parameters
image_dim = 784 #Image size is 28*28
gen_hidd_dim = 256 #No of hidden nodes
disc_hidd_dim = 256
z_noise_dim = 100

In [ ]:
#Xavier Initialization
def xavier_init(shape):
  return tf.random_normal(shape = shape, stddev=1./tf.sqrt(shape[0]/2.0))

In [ ]:
#Define weights and bias dictionaries
weights = {"disc_H": tf.Variable(xavier_init([image_dim, disc_hidd_dim])),
           "disc_final": tf.Variable(xavier_init([disc_hidd_dim, 1])),
           "gen_H": tf.Variable(xavier_init([z_noise_dim, gen_hidd_dim])),
           "gen_final": tf.Variable(xavier_init([gen_hidd_dim, image_dim]))}
           
bias = {"disc_H": tf.Variable(xavier_init([disc_hidd_dim])),
        "disc_final": tf.Variable(xavier_init([1])),
        "gen_H": tf.Variable(xavier_init([gen_hidd_dim])),
        "gen_final": tf.Variable(xavier_init([image_dim]))}

In [ ]:
#Creating Computational Graph
#Define Discriminator Function (Network)
def Discriminator(x):
  hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, weights["disc_H"]), bias["disc_H"]))
  final_layer = tf.add(tf.matmul(hidden_layer, weights["disc_final"]), bias["disc_final"])
  disc_output = tf.nn.sigmoid(final_layer)
  return disc_output

def Generator(z):
  hidden_layer = tf.nn.relu(tf.add(tf.matmul(z, weights["gen_H"]), bias["gen_H"]))
  final_layer = tf.add(tf.matmul(hidden_layer, weights["gen_final"]), bias["gen_final"])
  gen_output = tf.nn.sigmoid(final_layer)
  return gen_output

In [ ]:
#Define the placefolders for External Input
z_input = tf.placeholder(tf.float32, shape=[None, z_noise_dim], name="input_noise")
x_input = tf.placeholder(tf.float32, shape=[None, image_dim], name="real_input")

In [ ]:
#Building Generator Network
output_Gen = Generator(z_input)     # G(Z)

In [ ]:
#Building the Discriminator Network

real_output_Disc = Discriminator(x_input)      # D(X)
fake_output_Disc = Discriminator(output_Gen)    # D(G(Z))    

In [ ]:
Discriminator_Loss = -tf.reduce_mean(tf.log(real_output_Disc + 0.0001) + tf.log(1.-fake_output_Disc + 0.0001))

In [ ]:
Generator_Loss = -tf.reduce_mean(tf.log(fake_output_Disc + 0.0001))

In [ ]:
#Define Variables
Generator_var = [weights["gen_H"], weights["gen_final"], bias["gen_H"], bias["gen_final"]]
Discriminator_var = [weights["disc_H"], weights["disc_final"], bias["disc_H"], bias["disc_final"]]

In [ ]:
#Define the optimizer
Discriminator_optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(Discriminator_Loss, var_list= Discriminator_var)

Generator_optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(Generator_Loss, var_list=Generator_var)

In [ ]:
#Initialize the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  x_batch, _ = mnist.train.next_batch(batch_size)

  #Generate noise to feed to the discriminator
  z_noise = np.random.uniform(-1., 1., size=[batch_size, z_noise_dim])
  _, Disc_loss_epoch = sess.run([Discriminator_optimize, Discriminator_Loss], feed_dict={x_input: x_batch, z_input: z_noise})
  _, Gen_loss_epoch = sess.run([Generator_optimize, Generator_Loss], feed_dict={z_input: z_noise})

  if epoch%2000 == 0:
    print(f"Steps: {epoch}, Generator_Loss: {Gen_loss_epoch}, Discriminator_Loss: {Disc_loss_epoch}")

In [ ]:
#Testing 
#Generate Images from noise using the generator network
n=6
canvas = np.empty((28*n, 28*n))
for i in range(n):
  #Noise input
  z_noise = np.random.uniform(-1., 1., size=[batch_size, z_noise_dim])

  #Generate Input From Noise
  g = sess.run(output_Gen, feed_dict={z_input: z_noise})

  #Reverse colors for better display
  g  = -1 * (g-1)
  for j in range(n):
    #Draw the generated images
    canvas[i*28:(i+1)*28, j*28:(j+1)*28] = g[j].reshape([28,28])

plt.figure(figsize=(n,n))
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()   
sess.close()